Code to simulate Fibre Bragg Grating  
Written by: Samhita S Sodhi

This notebook contains code which loads spectral data of a particular gas and simulates Fibre Bragg Grating stretching/strain. The python script file "func.py" comprise of most of the functions that this notebook uses. 

Currently using Nicholas' data from SAIL labs to test and write the functions. 

Notes from the author:
- Nicholas' acetylene data does not appear to be the same as the spectra produced by NASA PSG https://psg.gsfc.nasa.gov/
- Code is written such that all the data files need to be loaded in, eg. gas and fibre bragg grating 
- Code has been checked to work with NASA PSG data files and Nicholas' SAIL lab files -- may need to edit if data format is changed 
- Formula for Gaussian used: amp*1/(sigma*np.sqrt(2*np.pi))*np.exp(-(x - mu)^2/(2*sigma^2)) source: https://www.geeksforgeeks.org/python-gaussian-fit/ , https://www.tutorialspoint.com/gaussian-fit-using-python, https://stackoverflow.com/questions/19206332/gaussian-fit-for-python
- Formula for Lorentzian used: (amp/np.pi) * (sigma/((x-mu)^2 + sigma^2)) source: https://lmfit.github.io/lmfit-py/builtin_models.html, https://scipython.com/book/chapter-8-scipy/examples/the-voigt-profile/
- The functions include: 
    - input_spectra: loads in the data, 
    - plot_spectra: simple plot of the data, 
    - fit_curves: fits a gaussian/lorentzian model to the datapoints, 
    - blackbodyabsorption: imprints a theoretical blackbody spectrum to data, 
    - apply_strain: simulates the stretching of fbg, 
    - plot_strainspectra: plots the output of simulate strain as a visualisation
    - correlation: for several values of strain plots the amount of light transmitted/reflected
    - convert_fluxunits_to_photoncounts: converts default flux units (erg / cm2 Hz s sr) to (photons/sec)

Issues
- Need to find appropriate lineshapes as lorentzian/gaussian don't fit data (example in scratch_SimulatingCode_FBG) -- try voigt with equation from HITRAN https://hitran.org/docs/definitions-and-units/,  https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.voigt_profile.html. Three methods of doing Voigt fitting - either convolve a Gaussian and Lorentzian, find a Voigt function online or write an equation http://emilygraceripka.com/blog/16
- The function which fits a Gaussian or Lorentzian to each peak uses a hardcoded "width value" which appears to work for the data I have worked with but may need to be looked into, there is a variable which contains the widths for each peak but does not appear to work
- Blackbody function converts the units into Angstroms because I was unsure about the units that astropy BlackBody requires as an input
- It appears Nicholas wants to convert the default astropy.BlackBody outputs using the scale he provided {bb = BlackBody(temperature=temp*u.K, scale=1.0 * u.J / (u.cm ** 2 * u.nm * u.s * u.sr))} but I believe this is an incorrect approach since the function requires the scale to be equal to (erg / cm2 Hz s sr) which (J / cm2 nm s sr) isn't, so I stick to the default units the package uses (erg / cm2 Hz s sr).
- Function that simulates the straining of the fibre bragg grating carried a bug which showed overfitting at the boundary (particularly for shorter wavelengths), this is likely due to extrapolation overfitting values at the boundary since it only had data from one side, this was corrected using bound_error = 'false' https://docs.scipy.org/doc/scipy/tutorial/interpolate/extrapolation_examples.html, https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html
- Theoretical modelling only uses blackbody -- improve by adding model for sunlight, whitelight, laser 
- The function used to simulate the straining of the fibre 'apply_strain', is still in this notebook since it depends on some of the previous variables, perhaps can be removed in the future. 
- The function which converts (erg / cm2 Hz s sr) to photon counts requires the wavelength and the transmitted values to have the same length but this causes the function 'correlation' to take a little longer than 1 hour to run which is not ideal and this will need to be looked into.
- When adding photon noise perhaps try Poisson noise rather than Gaussian noise as was used in Nicholas' code

In [ ]:
import sys
sys.path.insert(0, ".")

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from astropy import units as u
from astropy import constants as const
from astropy.modeling.models import BlackBody
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from funcs import *

**Preparing the input spectral data**

In [ ]:
spectra_wav, spectra_data = input_spectra('/home/samhitasodhi/Gas Sensor Project/Code - Nicholas Morley/fbg_prog/fbg_prog/Line_Lists/C2H2 gas cell-_40nm_1_T_C33-c2h2200C3M_42nm_0_T.txt', 0, 2, "\t", None, "nm", "dB")

In [ ]:
plot_spectra(spectra_wav, spectra_data)

In [ ]:
transmittance = fit_curves(spectra_wav, spectra_data, 0.09, 'lorentzian')

In [ ]:
flux, transmitted_intensity = blackbodyabsorption(spectra_wav, transmittance, 3000)
plot_spectra(spectra_wav, transmitted_intensity, yaxislabel = 'Transmitted Intensity (erg / cm2 Hz s sr)')

**Loading fibre bragg grating data**

In [ ]:
fibre_wavelength, fibregrating_line_data = input_spectra('/home/samhitasodhi/Gas Sensor Project/Code - Nicholas Morley/fbg_prog/fbg_prog/Line_Lists/FBG_C33-c2h2200C3M_42nm_0_T.txt', 0, 2, '\t', None, "nm", "dB")

In [ ]:
plot_spectra(fibre_wavelength, fibregrating_line_data)

In [ ]:
transmittanceFBG = fit_curves(fibre_wavelength, fibregrating_line_data, 0.09, 'lorentzian')

In [ ]:
fluxFBG, transmitted_intensityFBG = blackbodyabsorption(fibre_wavelength, transmittanceFBG, 3000)
plot_spectra(spectra_wav, transmitted_intensityFBG, title = 'Transmittance spectrum of acetylene fbg data', yaxislabel = 'Transmitted Intensity (erg / cm2 Hz s sr)')

**Simulating stretching of the fibre bragg grating**

In [ ]:
def apply_strain(strain, temp_change, thermal_exp_coeff, thermo_optic_coeff, strain_optic_coeff):
    """
    This function takes an input of a strain value and properties of the fibre bragg grating from which it 
    simulates the straining of the fibre bragg grating. 

    Parameters include:   
    - strain              : input a single strain value to stretch the fibre bragg grating given in micro-strain values i.e., XXe-6
    - temp_change         : change in temperature in degrees Celsius; 0 for constant temperature
    - thermal_exp_coeff   : thermal expansion coefficient for material of the fibre bragg grating per degree Celsius 
    - thermo_optic_coeff  : thermo-optic coefficient for material of the fibre bragg grating per degree Celsius
    - strain_optic_coeff  : effective strain-optic coefficient of the fibre 

    Equation used to simulate wavelength change: wav_shift = (1 - pe)*strain + (alpha + nu)*t_shift 
      (assumes stretching of the grating due to strain and temperature).
    """   

    t_shift = temp_change 
    alpha = thermal_exp_coeff
    nu = thermo_optic_coeff 
    pe = strain_optic_coeff 
    fillvalue = max(transmitted_intensity).value #chooses the max value rather than the boundary values because max_values are typically the boundary values

    wav_shift = (1 - pe)*strain + (alpha + nu)*t_shift
    fibre_wavelength_new = spectra_wav + wav_shift*spectra_wav #creates new wavelengths for which there is no existing data 
    #fillvalue = 5.075295546167134e-06
    f1 = interp1d(fibre_wavelength_new, transmitted_intensityFBG, bounds_error = False, fill_value = fillvalue) #bounds_error = False -> interp1d sets the out-of-range values with the fill_value, which is nan by default / here fitted to the max value
    interpolatedvals = f1(fibre_wavelength)

    return fibre_wavelength_new, interpolatedvals

In [ ]:
def plot_strainspectra(xlimits = None, ylimits = None):
    """
    This function plots the spectra once the strain has been applied to the fibre bragg grating. 
    
    Parameters include: 
      - xlimits  : user has a choice to input a range of x values (wavelength) i.e.,  [ , ] for the plot else default is complete range of values 
      - ylimits  : user has a choice to input a range of y values (Transmitted Intensity) i.e.,  [ , ] for the plot else default is complete range of values
    """
    # Plotting different curves 
    if xlimits is not None or ylimits is not None: 
      # Plot with custom limits
      figure(figsize=(15, 4))
      plt.plot(spectra_wav, transmitted_intensity, c = 'navy', label = 'Original gas spectra')
      plt.plot(fibre_wavelength, transmitted_intensityFBG, c = 'powderblue', label = 'Original FBG spectra')
      plt.plot(fibre_wavelength_new, interpolatedvals, c = 'red', label = 'Strained FBG spectra')
      plt.xlim(xlimits)
      plt.ylim(ylimits)
      plt.legend(loc = 'lower right')
      plt.title('Straining the Fiber Bragg Grating')
      plt.xlabel('Wavelength (um)')
      plt.ylabel('Transmitted Intensity (erg / cm2 Hz s sr)')
      plt.show()
      
    else: 
      # Plot without custom limits
      figure(figsize=(15, 4))
      plt.plot(spectra_wav, transmitted_intensity, c = 'navy', label = 'Original gas spectra')
      plt.plot(fibre_wavelength, transmitted_intensityFBG, c = 'powderblue', label = 'Original FBG spectra')
      plt.plot(fibre_wavelength_new, interpolatedvals, c = 'red', label = 'Strained FBG spectra')
      plt.legend(loc = 'lower right')
      plt.title('Straining the Fiber Bragg Grating')
      plt.xlabel('Wavelength (um)')
      plt.ylabel('Transmitted Intensity (erg / cm2 Hz s sr)')
      plt.show()
    
    return

In [ ]:
#testing the straining function; works well shows that no only is the fibre bragg grating being stretched but distances between the peaks are also changing

fibre_wavelength_new, interpolatedvals = apply_strain(0.0005, 0, 0.55e-6, 8.6e-6, 0.22) #values for silica, no temperature change

In [ ]:
plot_strainspectra()

In [ ]:
def correlation(strainvalues, normalisation = 'True'):
    """
    For different input values of strain this function plots the amount of total, transmitted and reflected light once incident light
    passes through the fibre bragg grating.

    Parameters include: 
    - strainvalues  : user inputs values of strain i.e., strain = np.linspace(0, 0.0014, 500) for which the stretching of the fibre bragg 
                      grating is simulated
    - normalisation : user can choose whether to normalise the calculations to see the outputs as a fraction rather than true values else 
                      default is 'True'. 
    """

    reflectedvals = []
    transmittedvals = []
    totallightvals = []
    strain = strainvalues
    fillvalue = max(transmitted_intensity).value  

    for i in strain:
        fibre_wavelength_new, interpolatedvals = apply_strain(i, 0, 0.55e-6, 8.6e-6, 0.22) # values for silica, no temperature change
        totallight = fillvalue*np.ones(len(interpolatedvals*spectra_data)) # calculates the total amount of light going through the system
        reflected = (totallight - interpolatedvals*spectra_data).sum() # reflected light = total - transmitted
        transmitted = np.array(interpolatedvals*spectra_data).sum() # transmitted light
        
        sumtotal_light = totallight.sum()

        if normalisation == 'True':
            normalise_transmitted = transmitted / sumtotal_light #normalises transmitted light
            normalise_reflected = reflected / sumtotal_light #normalises reflected light
            
            transmittedvals.append(normalise_transmitted)
            reflectedvals.append(normalise_reflected)
            totallightvals.append(normalise_transmitted + normalise_reflected)
            
        elif normalisation == 'False':
            transmittedvals.append(transmitted)
            reflectedvals.append(reflected)
            totallightvals.append(transmitted+reflected) 
            
        else: 
            print("Please type either 'True' or 'False'")

    if normalisation == 'True':
        figure(figsize=(15, 4))
        plt.plot(strain, transmittedvals)
        plt.title('Transmitted light')
        plt.xlabel('Strain')
        plt.ylabel('Light Transmitted as a fraction of the total light')
        plt.show()

        figure(figsize=(15, 4))
        plt.plot(strain, reflectedvals)
        plt.title('Reflected light')
        plt.xlabel('Strain')
        plt.ylabel('Light Reflected as a fraction of the total light')
        plt.show()  
        
        figure(figsize=(15, 4))
        plt.plot(strain, totallightvals)
        plt.title('Total light')
        plt.xlabel('Strain')
        plt.ylabel('Total Light')
        plt.show()

    elif normalisation == 'False':
        figure(figsize=(15, 4))
        plt.plot(strain, transmittedvals)
        plt.title('Transmitted light')
        plt.xlabel('Strain')
        plt.ylabel('Light Transmitted (erg / cm2 Hz s sr)')
        plt.show()

        figure(figsize=(15, 4))
        plt.plot(strain, reflectedvals)
        plt.title('Reflected light')
        plt.xlabel('Strain')
        plt.ylabel('Light Reflected (erg / cm2 Hz s sr)')
        plt.show()  
        
        figure(figsize=(15, 4))
        plt.plot(strain, totallightvals)
        plt.title('Total light')
        plt.xlabel('Strain')
        plt.ylabel('Total Light (erg / cm2 Hz s sr)')
        plt.show()

    return transmittedvals, reflectedvals

In [ ]:
strain = np.linspace(0, 0.0016, 16363) # takes a long time to run, else conversion to photoncounts does not work - wav, data different sizes! 
transmittedlight, reflectedlight = correlation(strain, 'False')

totallight = np.array(transmittedlight) + np.array(reflectedlight)
print(totallight[0])

# strain = np.linspace(0, 0.0016, 500) 
# transmittedlightNorm, reflectedlightNorm = correlation(strain, 'True')

**Converting units to photon counts**

In [ ]:
#testing unit conversion function
photoncount_persec = convert_fluxunits_to_photoncounts(spectra_wav, transmittedlight, 0.35, 0.000000503, 0.045)
photoncount_persec = convert_fluxunits_to_photoncounts(spectra_wav, reflectedlight, 0.35, 0.000000503, 0.045)